I ran this notebook locally.

## Move to correct directory

In [1]:
import os
current_file_dir = os.getcwd()
os.chdir("../../codes/")

In [2]:
import tqdm
import pandas as pd
from torchvision import transforms

# Setting

Get possible models etc.

In [3]:
from src.datasets.imagenette2_class_to_id_mapping import imagenette2_id_to_class_mapping

In [4]:
from src import strings_to_classes_mappings
from src.utils import create_classes_from_strings

print("Possible datasets:", list(strings_to_classes_mappings.datasets_mapping.keys()))
print("Possible models:", list(strings_to_classes_mappings.models_mapping.keys()))

Possible datasets: ['imagenette2']
Possible models: ['ConvNeXtV2_Nano', 'DeiT_S', 'DeiT_T', 'DenseNet121', 'EfficientNet_B3', 'EfficientNet_B4', 'MobileNetV3', 'PyramidViT_V2_B2', 'Res2Net50', 'ResNet18', 'ResNet50', 'ResNeXt50', 'Swin_T', 'ViT_B_32']


Set parameters of the analysis:

In [10]:
def compute_accuracy(models_list, dataset_name, explanation_name, number_of_batches_to_process, batch_size, shuffle, dataset_split, root_images, root_explanations, device, models_weigths_pretrained, is_transform_model_specific):
    results_dict = {}
    
    for model_name in models_list:
        dataset_manager, model, explanation = create_classes_from_strings(
                    model_name, dataset_name, explanation_name, 
                    root_images + dataset_split + '/', root_explanations, device,
                    models_weigths_pretrained)
        model = model.get_model()
        model = model.to(device)

        dataloader = dataset_manager.get_dataloader(model=model, 
                batch_size=batch_size,
                shuffle=shuffle)
        
        if not is_transform_model_specific:
            # overwrite dataloader
            dataloader.dataset.transform = transforms.Compose([
                    transforms.Resize((224, 224)),
                    transforms.ToTensor(),
                    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
                ])
        
        processed_images = 0
        if number_of_batches_to_process is None:
            number_of_batches_to_process = len(dataloader)
        number_of_batches_to_process = min(number_of_batches_to_process, len(dataloader))
        images_to_process_limit = min(len(dataloader.dataset), number_of_batches_to_process * dataloader.batch_size)

        pbar = tqdm.tqdm(dataloader, desc='Computing predictions for batches', total=number_of_batches_to_process)

        predicted_classes = []
        real_classes = []    

        for images, labels, paths in pbar:

            if processed_images >= images_to_process_limit:
                break

            images = images.to(device)
            predicted_vectors = model(images)

            predicted_class_ids = [int(x.argmax()) for x in predicted_vectors]
            predicted_classes += [imagenette2_id_to_class_mapping[x] for x in predicted_class_ids]
            real_class_ids = labels
            real_classes += [imagenette2_id_to_class_mapping[int(x)] for x in real_class_ids]

            processed_images += len(images)

        model = model.to('cpu')
        del model, dataset_manager, explanation, dataloader

        accuracy = round(sum([1 for x, y in zip(predicted_classes, real_classes) if x == y]) / len(real_classes),4)
        print(f'accuracy: {accuracy:.3f} \t - {model_name}')
        results_dict[model_name] = {'accuracy': accuracy}

    df = pd.DataFrame(results_dict).T
    df.index.name = 'model'
    per_model_or_not_filename_part = 'model_specific' if is_transform_model_specific else 'all_same'
    df.to_csv(f'{current_file_dir}/accuracy_VIT_{per_model_or_not_filename_part}_transforms-{dataset_split}_{number_of_batches_to_process * batch_size}.csv')
    return df

## Computing accuracy

In [11]:
parameters = {
    "models_list": ['ViT_B_32'],
    "dataset_name": 'imagenette2',
    "explanation_name": 'IntegratedGradients',
    "number_of_batches_to_process": 2,
    "batch_size": 2, 
    "shuffle": True,
    "dataset_split": 'val',
    "root_images": f'../datasets/imagenette2/',
    "root_explanations": current_file_dir,
    "device": "cuda",
    "models_weigths_pretrained": "imagenet",
    "is_transform_model_specific": True,
}

In [12]:
parameters['number_of_batches_to_process'] = 512
parameters['batch_size'] = 2

---

In [16]:
parameters['dataset_split'] = 'train'
parameters['is_transform_model_specific'] = True
compute_accuracy(**parameters)

Loaded default imagenet-pretrained model: https://huggingface.co/timm/vit_base_patch32_224.augreg_in21k_ft_in1k


Computing predictions for batches: 100%|██████████| 512/512 [01:21<00:00,  6.31it/s]


accuracy: 0.884 	 - ViT_B_32


,accuracy
model,
ViT_B_32,0.8838


In [17]:
parameters['dataset_split'] = 'val'
parameters['is_transform_model_specific'] = True
compute_accuracy(**parameters)

Loaded default imagenet-pretrained model: https://huggingface.co/timm/vit_base_patch32_224.augreg_in21k_ft_in1k


Computing predictions for batches: 100%|██████████| 512/512 [02:28<00:00,  3.45it/s]


accuracy: 0.894 	 - ViT_B_32


,accuracy
model,
ViT_B_32,0.8945


In [14]:
parameters['dataset_split'] = 'train'
parameters['is_transform_model_specific'] = False
compute_accuracy(**parameters)

Loaded default imagenet-pretrained model: https://huggingface.co/timm/vit_base_patch32_224.augreg_in21k_ft_in1k


Computing predictions for batches: 100%|██████████| 512/512 [00:57<00:00,  8.98it/s]


accuracy: 0.857 	 - ViT_B_32


,accuracy
model,
ViT_B_32,0.8574


In [15]:
parameters['dataset_split'] = 'val'
parameters['is_transform_model_specific'] = False
compute_accuracy(**parameters)

Loaded default imagenet-pretrained model: https://huggingface.co/timm/vit_base_patch32_224.augreg_in21k_ft_in1k


Computing predictions for batches: 100%|██████████| 512/512 [00:57<00:00,  8.89it/s]


accuracy: 0.865 	 - ViT_B_32


,accuracy
model,
ViT_B_32,0.8652
